In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

# Select variable

In [296]:
#Variable 
var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
#var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length" # This option doesnt work because there is not seasonal data
#var ="DJC" # This option doesnt work because there is not seasonal data


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")


filesSeas45 =  glob.glob(folder+"*rcp45_"+var+"_seasonal.nc")
filesSeas85 =  glob.glob(folder+"*rcp85_"+var+"_seasonal.nc")



# FOR RCP 45

In [297]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

ValueError: must supply at least one object to concatenate

## Getting time windows and percentiles

In [ ]:
# Table 
# Mapping data - 30 year means
tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

In [ ]:
%time dfperT45 = perc30yavgTable.drop('realization').to_dataframe().dropna()

In [ ]:
dfperT45['season'] = "annual"

In [ ]:
dfperT45

# FOR RCP85

In [ ]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Getting time windows and percentiles

In [ ]:
#for (file, percentile, season) in ((files45, 45, "annual"), (files85, 85, "annual"),
#                                  (filesSeas45, 45, )):
# Table 
# Mapping data - 30 year means
tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable85 = ens.ensemble_percentiles(ds30yavgTable)
#perc30yavgTable 

%time dfperT85 = perc30yavgTable85.drop('realization').to_dataframe().dropna()
dfperT85['season'] = "annual"

dfperT85

# FOR Seasonal RCP45

## Loop to create all 4 dataframes

In [ ]:
def load_seasonal_data(files, scenario):
    dsEns= ens.create_ensemble(files)
    dsEns.time.dt.season[0:4]
    # Seasonal example 30 y means
    listds = []
    i=0
    for s in dsEns.time.dt.season[0:4]:
        print (i)
        tmp1 = dsEns.sel(time=(dsEns.time.dt.year>=1981))
        tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
        window = 30
        time1 = tmp1.time[0::window]
        # 30 y means
        ds30yavgTable = tmp1.coarsen(time=window).mean()
        ds30yavgTable['time'] = time1
        # percentiles
        perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
        #perc30yavgTable 
        dfSeas = perc30yavgTable.drop('realization').to_dataframe().dropna()
        if i== 0: season = "winter" 
        elif i == 1: season = "spring"
        elif i == 2: season = "summer"
        elif i == 3: season = "fall"
        print (season)
        dfSeas["season"] = season
        print(dfSeas)
        dfSeas["scenario"] = scenario        
        i = i+1
        listds.append(dfSeas)
    return pd.concat(listds)

## To split by region FOR RCP45 AND RCP85, Annual and seasonal

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
 def strings_to_nan(col):
    return col.map(lambda x: np.nan if isinstance(x, str) else x)

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [ ]:
dfperT45["scenario"] = 45
dfperT85["scenario"] = 85

In [ ]:
dfbig = pd.concat((load_seasonal_data(filesSeas45, 45), load_seasonal_data(filesSeas85,  85),
                   dfperT45, dfperT85))
dfbig = dfbig.reset_index()
dfbig["time"] = dfbig.time.dt.year

In [ ]:
if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
    dfbig[var+"_p10"] = dfbig[var+"_p10"] -273.15
    dfbig[var+"_p50"] = dfbig[var+"_p50"] -273.15
    dfbig[var+"_p90"] = dfbig[var+"_p90"] -273.15
if (var == "solidprcptot"):
    dfbig[var+"_p10"] = dfbig[var+"_p10"] /10
    dfbig[var+"_p50"] = dfbig[var+"_p50"] /10
    dfbig[var+"_p90"] = dfbig[var+"_p90"] /10
dfbig

In [298]:
for region, (name, short) in short_dict.items():
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    df_per_region = pd.merge(dfregion, dfbig.reset_index(), on=["lat","lon"])
    df_subregion_means = df_per_region.groupby(["time", name, "season","scenario"]).mean()[[var+"_p10",var+"_p50", var+"_p90"]]
    df_subregion_means[var+"_p10"] = round(df_subregion_means[var+"_p10"],2)
    df_subregion_means[var+"_p50"] = round(df_subregion_means[var+"_p50"],2)
    df_subregion_means[var+"_p90"] = round(df_subregion_means[var+"_p90"],2)
    df_subregion_means[var+"_p1090"] = df_subregion_means[var+"_p10"].map(str)+" - "+df_subregion_means[var+"_p90"].map(str)
    df_subregion_means_perc = pd.wide_to_long(df_subregion_means.reset_index(), var+"_p", i=("time","season","scenario",name), j="percentility")
    df_subregion_means_perc = df_subregion_means_perc.rename(columns={var+"_p":var })
    for (sr,df) in df_subregion_means_perc.reset_index().groupby(name):
        df = df.copy()
        df["sp"] = df.season + df.percentility.map(str)
        df["time_scenario"] = df.time.map(str) + df.scenario.map(str)    
        dftable = pd.pivot(df,columns="time_scenario", index="sp", values=var)
        print ("9/11 Reindexing table")
        dftable2 = dftable.reindex(index=["annual50", "annual1090","winter50","winter1090","spring50","spring1090",
                           "summer50","summer1090","fall50","fall1090"])
        dftable2.index=["Annuel", "Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
                           "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]
        print("10/11 Getting mean for historic data")
        dftable2["1981-2010"] = ((strings_to_nan(dftable2.iloc[:,0]) + strings_to_nan(dftable2.iloc[:,1])) / 2).fillna("")
        dftable3 = dftable2.drop(columns=["198145","198185",'201145', '201185'])
        dftable4 = dftable3[["1981-2010", "204145","204185", "207145", "207185"]]
        dftable5 = dftable4.rename(columns={'204145': 'Emissions moderees 2050',
                                            '204185': 'Emissions Elevees 2050',
                                            '207145': 'Emissions moderees 2080',
                                            '207185': 'Emissions Elevees 2080', })
        print("11/11 ... CREATING CSV!!")
        dftable5.to_csv(replace_all(sr+"_"+var+"_table.csv", d))

KeyError: "None of [Index(['DJC_p10', 'DJC_p50', 'DJC_p90'], dtype='object')] are in the [columns]"

In [ ]:
dftable5